In [1]:
import pandas as pd
import plotly.express as px

from utils import get_connection

db = get_connection("records")['db']
works = db["works"]

In [2]:
title = "Citations per publication"
df = pd.DataFrame(works.find({}, {"_id": 0, "cited_by_count": 1}))
fig = px.histogram(df, x="cited_by_count", nbins=100, title=title)
fig.show()

In [3]:
title = "Citations per publication (log)"
df = pd.DataFrame(works.find({}, {"_id": 0, "cited_by_count": 1}))
fig = px.histogram(df, x="cited_by_count", nbins=100, log_y=True, color_discrete_sequence=["teal"], title=title)
fig.show()

In [4]:
title = "Publications with no citations"
df = pd.DataFrame(works.find({"cited_by_count": {"$eq": 0}}, {"_id": 0, "cited_by_count": 1, "publication_year": 1}))
fig = px.histogram(df, x="publication_year", text_auto=True, title=title)
fig.update_layout(bargap=0.1, xaxis={"dtick": 1})
fig.show()

In [5]:
title = "Publication count throughout the years"
df = pd.DataFrame(works.find({}, {"_id": 0, "publication_year": 1}))
fig = px.histogram(df, x="publication_year", text_auto=True, title=title)
fig.update_layout(bargap=0.1, xaxis={"dtick": 1})
fig.show()

In [6]:
title = "Citations per publication (less than 100 citations)"
df = pd.DataFrame(works.find({ "cited_by_count": { "$lt": 100 } }, {"_id": 0, "cited_by_count": 1}))
fig = px.histogram(df, x="cited_by_count", nbins=30, text_auto=True, title=title)
fig.show()

In [7]:
title = "Number of authors per publication"
authors = list(works.aggregate([
    {
        "$project": {
            "_id": 0,
            "authorsCount":  { "$size": "$authorships" }
        }
    }
]))

df = pd.DataFrame(authors)
fig = px.histogram(df, x="authorsCount", nbins=50, text_auto=True, title=title)
fig.update_layout(xaxis_title_text="Number of authors")
fig.show()

In [8]:
title = "Number of authors vs citations"
authors = works.aggregate([
    {
        "$project": {
            "_id": 0,
            "authorsCount":  { "$size": "$authorships" },
            "cited_by_count": 1,
            "type": 1
        }
    }
])

df = pd.DataFrame(authors)
fig = px.scatter(df, height=700, opacity=0.7, x="authorsCount", y="cited_by_count", color="type", symbol="type", title=title)
fig.update_layout(xaxis_title_text="Number of authors", yaxis_title_text="Citation count")
fig.show()

In [9]:
title = "Number of authors vs citations (1-20 authors)"
authors = works.aggregate([
    {
        "$project": {
            "_id": 0,
            "authorsCount":  { "$size": "$authorships" },
            "cited_by_count": 1,
            "type": 1
        }
    },
    {
        "$match": {
            "authorsCount": { "$lte": 20 }
        }
    }
])

df = pd.DataFrame(authors)
fig = px.scatter(df, height=1000, opacity=0.5, x="authorsCount", y="cited_by_count", color="type", symbol="type", title=title)
fig.update_layout(xaxis_title_text="Number of authors", yaxis_title_text="Citation count")
fig.update_traces(marker_size=10)
fig.show()

In [10]:
title = "Number of authors vs citations (1-20 authors, article excluded)"
authors = works.aggregate([
    {
        "$project": {
            "_id": 0,
            "authorsCount":  { "$size": "$authorships" },
            "cited_by_count": 1,
            "type": 1
        }
    },
    {
        "$match": {
            "authorsCount": { "$lte": 20 },
            "type": { "$ne": "article" }
        }
    }
])

df = pd.DataFrame(authors)
fig = px.scatter(df, height=700, opacity=0.7, x="authorsCount", y="cited_by_count", color="type", symbol="type", title=title)
fig.update_layout(xaxis_title_text="Number of authors", yaxis_title_text="Citation count")
fig.update_traces(marker_size=10)
fig.show()

In [11]:
title = "Top 20 authors (all affiliations; not just DK)"
authors = works.aggregate([
    {
        "$unwind": "$authorships"
    },
    {
        "$group": {
            "_id": "$authorships.raw_author_name",
            "count": { "$sum": 1 }
        }
    },
    {
        "$sort": {
            "count": -1
        }
    },
    {
        "$limit": 20
    }
])

df = pd.DataFrame(authors)
fig = px.bar(df, height=700, y="_id", x="count", text_auto=True, title=title)
fig.update_layout(xaxis_title_text="Publication count", yaxis_title_text="", yaxis={"autorange": "reversed"})
fig.show()

In [12]:
import json, requests

title = "Top 10 most cited works in the related works"
related = works.aggregate([
    {
        "$unwind": "$related_works"
    },
    {
        "$group": {
            "_id": "$related_works",
            "count": { "$sum": 1 }
        }
    },
    {
        "$sort": {
            "count": -1
        }
    },
    {
        "$limit": 10
    }
])

# quick workaround to get names to IDs
def get_full_info(work):
    api_url = work["_id"].replace("https://", "https://api.")
    r = requests.get(api_url)
    info = r.json()

    return {
        "_id": f"{info['title']} ({work['_id']})",
        "count": work["count"]
    }

df = pd.DataFrame([get_full_info(work) for work in list(related)])
fig = px.bar(df, y="_id", x="count", text_auto=True, title=title)
fig.update_layout(xaxis_title_text="", yaxis_title_text="", yaxis={"autorange": "reversed"})
fig.show()